In [1]:
# multi_imu_headturn.py
# pip install requests pyttsx3 numpy

import time
import math
import threading
import queue
import atexit
import csv
import datetime as dt
from collections import deque
from typing import Callable, Optional, Deque, Dict, Any, Tuple

import numpy as np
import requests

In [2]:

# ================================
# CONFIG: two IMUs
# ================================
SENSORS = [
    {
        "name": "One",
        "url": "http://192.168.23.4:8080/get?gyrX&gyr_time&gyrY&gyrZ",
        "log_csv": "imu_helmetA_50hz.csv",
    },
    {
        "name": "Two",
        "url": "http://192.168.23.136:8080/get?gyrX&gyr_time&gyrY&gyrZ",
        "log_csv": "imu_helmetB_50hz.csv",
    },
]

PERIOD_S = 0.020          # 50 Hz
SPEAK_SOURCE_NAME = True  # True → say "HelmetA left turn", False → just "Left turn"



In [3]:

import threading, queue
from typing import Optional, Tuple

def _say_once(text: str, rate=180, volume=1.0, voice_contains="en"):
    import pyttsx3
    engine = pyttsx3.init()
    engine.setProperty("rate", rate)
    engine.setProperty("volume", volume)
    try:
        for v in engine.getProperty("voices"):
            if voice_contains and voice_contains.lower() in str(v.id).lower():
                engine.setProperty("voice", v.id)
                break
    except Exception:
        pass
    engine.say(text)
    engine.runAndWait()      # completes this one utterance [web:31]
    try:
        engine.stop()
    except Exception:
        pass
    del engine

class SpeechWorker:
    def __init__(self, rate: int = 180, volume: float = 1.0, voice_contains: str = "en"):
        self._q: "queue.Queue[Optional[Tuple[str, Optional[threading.Event]]]]" = queue.Queue()
        self._closed = threading.Event()
        self._rate, self._volume, self._voice_contains = rate, volume, voice_contains
        self._thread = threading.Thread(target=self._run, daemon=True)
        self._thread.start()

    def _run(self):
        while True:
            item = self._q.get()
            try:
                if item is None:
                    break
                text, done_evt = item
                try:
                    _say_once(text, self._rate, self._volume, self._voice_contains)  # robust per-utterance init [web:21][web:25]
                except Exception as e:
                    print(f"[TTS worker] speak failed: {e}")
                finally:
                    if done_evt is not None:
                        done_evt.set()
            finally:
                self._q.task_done()

    def speak(self, text: str, block: bool = False, timeout: Optional[float] = None):
        if self._closed.is_set():
            return
        done = threading.Event() if block else None
        self._q.put((text, done))
        if block and done is not None:
            done.wait(timeout=timeout)

    def close(self, wait: bool = True):
        if not self._closed.is_set():
            self._closed.set()
            self._q.put(None)
            if wait:
                self._thread.join(timeout=3.0)

# Example
speaker = SpeechWorker(rate=250, volume=3.0, voice_contains="en")

In [4]:
# FusionCoordinator (dedupe speech)

class FusionCoordinator:
    """
    Simple deduplicator: if we say the same phrase again within min_interval seconds,
    skip the duplicate.
    """
    def __init__(self, min_interval: float = 0.8):
        self.min_interval = float(min_interval)
        self._last_said: Dict[str, float] = {}
        self._lock = threading.Lock()

    def maybe_speak(self, phrase: str):
        now = time.monotonic()
        with self._lock:
            last = self._last_said.get(phrase, 0.0)
            if (now - last) >= self.min_interval:
                self._last_said[phrase] = now
                speaker.speak(phrase)


fusion = FusionCoordinator(min_interval=0.6)

In [10]:

# ================================
# Real-time head-turn detector
# ================================
class RealTimeHeadTurnDetector:
    RIGHT, STATIC, LEFT = -1, 0, +1
    label_name = {RIGHT: "RIGHT_TURN", STATIC: "STATIC", LEFT: "LEFT_TURN"}

    def __init__(
        self,
        on_event: Callable[[dict], None],
        lp_cut_hz: float = 3.0,
        slope_win_sec: float = 0.40,
        rate_hi_deg_s: float = 6.0,
        rate_lo_deg_s: float = 2.0,
        static_max_deg_s: float = 1.5,
        min_turn_dur: float = 0.60,
        min_static_dur: float = 1,    #1.20
        min_turn_angle_deg: float = 35.0,
        drift_guard_deg_s: float = 0.02,
        front_leave_deg: float = 25.0,
        front_return_deg: float = 10.0,
        front_min_hold_s: float = 0.4, #0.5,
    ):
        self.on_event = on_event
        # thresholds
        self.lp_cut_hz = lp_cut_hz
        self.slope_win_sec = slope_win_sec
        self.RATE_HI = rate_hi_deg_s
        self.RATE_LO = rate_lo_deg_s
        self.STATIC_MAX = static_max_deg_s
        self.MIN_TURN_DUR = min_turn_dur
        self.MIN_STATIC_DUR = min_static_dur
        self.MIN_TURN_ANGLE = min_turn_angle_deg
        self.DRIFT_GUARD = drift_guard_deg_s
        self.FRONT_LEAVE_DEG = front_leave_deg
        self.FRONT_RETURN_DEG = front_return_deg
        self.FRONT_MIN_HOLD = front_min_hold_s

        # state
        self.prev_ts: Optional[float] = None
        self.prev_w_f: float = 0.0   # filtered gyro (rad/s)
        self.yaw_deg: float = 0.0    # integrated yaw (deg)
        self.state: int = self.STATIC
        self.state_start_ts: Optional[float] = None
        self.yaw_at_state_start: float = 0.0

        self.win_t: Deque[float] = deque()
        self.win_yaw: Deque[float] = deque()

        # front-reference logic
        self.front_ref_deg: float = 0.0
        self.away_from_front: bool = False
        self.front_hold_start: Optional[float] = None

    # ---- helpers ----
    @staticmethod
    def _iir_alpha(dt, cut_hz):
        if cut_hz <= 0 or dt <= 0:
            return 1.0
        rc = 1.0 / (2 * math.pi * cut_hz)
        return dt / (rc + dt)

    @staticmethod
    def _linregress_slope(x: np.ndarray, y: np.ndarray) -> float:
        n = x.size
        if n < 3:
            return 0.0
        xm, ym = x.mean(), y.mean()
        denom = float(np.sum((x - xm) ** 2))
        if denom <= 0.0:
            return 0.0
        return float(np.sum((x - xm) * (y - ym)) / denom)

    def _emit_event(self, s: float, e: float, code: int, net_angle: float):
        ev = {
            "start_s": round(s, 3),
            "end_s": round(e, 3),
            "duration_s": round(e - s, 3),
            "label": self.label_name[code],
            "net_angle_deg": round(float(net_angle), 2),
        }
        try:
            self.on_event(ev)
        except Exception as ex:
            print(f"[Detector] on_event error: {ex}")

    def _emit_front_event(self, s: float, e: float):
        ev = {
            "start_s": round(s, 3),
            "end_s": round(e, 3),
            "duration_s": round(e - s, 3),
            "label": "BACK_TO_FRONT",
            "net_angle_deg": round(float(self.yaw_deg - self.front_ref_deg), 2),
        }
        try:
            self.on_event(ev)
        except Exception as ex:
            print(f"[Detector] on_event error: {ex}")

    def _window_max_abs_slope(self) -> float:
        x = np.asarray(self.win_t, float)
        y = np.asarray(self.win_yaw, float)
        if x.size < 4:
            return 0.0
        k = max(3, x.size // 6)
        vals = []
        for i0 in range(0, x.size - k, k):
            i1 = i0 + k
            vals.append(abs(self._linregress_slope(x[i0:i1], y[i0:i1])))
        return max(vals) if vals else 0.0

    def recenter_front(self):
        """Optional: call to say 'this yaw is new front direction'."""
        self.front_ref_deg = self.yaw_deg
        self.away_from_front = False
        self.front_hold_start = None

    # ---- core update ----
    def update(self, ts: float, gyrY_rad_s: float):
        if self.prev_ts is None:
            self.prev_ts = ts
            self.prev_w_f = gyrY_rad_s
            self.state_start_ts = ts
            self.yaw_at_state_start = self.yaw_deg
            self.win_t.clear()
            self.win_yaw.clear()
            self.win_t.append(ts)
            self.win_yaw.append(self.yaw_deg)
            return

        dt = max(1e-6, ts - self.prev_ts)

        # 1) IIR LP filtered gyro
        a = self._iir_alpha(dt, self.lp_cut_hz)
        w_f = self.prev_w_f + a * (gyrY_rad_s - self.prev_w_f)

        # drift guard in deg/s
        if abs(math.degrees(w_f)) < self.DRIFT_GUARD:
            w_f = 0.0

        # 2) integrate to yaw (deg)
        yaw_rad = math.radians(self.yaw_deg) + 0.5 * (self.prev_w_f + w_f) * dt
        self.yaw_deg = math.degrees(yaw_rad)
        self.prev_w_f = w_f
        self.prev_ts = ts

        # 3) slope window buffer
        self.win_t.append(ts)
        self.win_yaw.append(self.yaw_deg)
        while len(self.win_t) >= 2 and (self.win_t[-1] - self.win_t[0]) > self.slope_win_sec:
            self.win_t.popleft()
            self.win_yaw.popleft()

        slope = self._linregress_slope(np.asarray(self.win_t), np.asarray(self.win_yaw))

        # 4) FSM left/right/static
        if self.state == self.STATIC:
            if slope <= -self.RATE_HI:
                self._finalize_span_if_valid(ts, self.RIGHT)
            elif slope >= +self.RATE_HI:
                self._finalize_span_if_valid(ts, self.LEFT)
        elif self.state == self.LEFT:
            if slope <= +self.RATE_LO:
                self._finalize_span_if_valid(ts, self.STATIC)
        elif self.state == self.RIGHT:
            if slope >= -self.RATE_LO:
                self._finalize_span_if_valid(ts, self.STATIC)

        # 5) FRONT_RETURN detection
        yaw_rel = self.yaw_deg - self.front_ref_deg
        abs_yaw = abs(yaw_rel)

        # mark as "away" once leave threshold crossed
        if not self.away_from_front and abs_yaw >= self.FRONT_LEAVE_DEG:
            self.away_from_front = True
            self.front_hold_start = None

        # if away, check for return window and flatness
        if self.away_from_front and abs_yaw <= self.FRONT_RETURN_DEG:
            if abs(slope) <= self.STATIC_MAX:
                if self.front_hold_start is None:
                    self.front_hold_start = ts
                elif (ts - self.front_hold_start) >= self.FRONT_MIN_HOLD:
                    # we are back to front
                    self._emit_front_event(self.front_hold_start, ts)
                    self.away_from_front = False
                    self.front_hold_start = None
            else:
                self.front_hold_start = None
        else:
            self.front_hold_start = None
    
    """
    Raw Gyro → Low-Pass Filter → Drift Removal → Integration → Head Angle
         ↓
    Head Angle → Sliding Window → Slope Calculation → State Machine
         ↓
    Head Angle → Front Reference Check → Return Detection
    """

    def _finalize_span_if_valid(self, end_ts: float, next_state: int):
        if self.state_start_ts is None:
            self.state = next_state
            self.state_start_ts = end_ts
            self.yaw_at_state_start = self.yaw_deg
            return

        s = self.state_start_ts
        e = end_ts
        dur = e - s
        code = self.state
        net_angle = self.yaw_deg - self.yaw_at_state_start

        if code == self.STATIC:
            if dur >= self.MIN_STATIC_DUR and self._window_max_abs_slope() <= self.STATIC_MAX:
                self._emit_event(s, e, code, net_angle)
        else:
            if (dur >= self.MIN_TURN_DUR and abs(net_angle) >= self.MIN_TURN_ANGLE
                    and math.copysign(1.0, net_angle) == code):
                self._emit_event(s, e, code, net_angle)

        self.state = next_state
        self.state_start_ts = end_ts
        self.yaw_at_state_start = self.yaw_deg

    def flush(self, ts_now: Optional[float] = None):
        if self.state_start_ts is None:
            return
        end_ts = ts_now if ts_now is not None else (self.prev_ts or time.time())
        self._finalize_span_if_valid(end_ts, self.state)


[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208526B5730>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852B76E80>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852950F70>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852B76EE0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529F6430>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BD6C70>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

In [6]:

# ================================
# ActionTracker: complete turn cycles
# ================================
class ActionTracker:
    """
    Higher-level 'complete action' detector for each helmet.

    A TURN_CYCLE is detected if, within a sliding window (window_s):
      - EITHER: ≥2 LEFT_TURN and ≥2 RIGHT_TURN (any order)
      - OR:     ≥2 BACK_TO_FRONT

    'subtype' tells which pattern triggered:
      - "LR_MIX"        : left/right mix
      - "DOUBLE_RETURN" : two BACK_TO_FRONT
    """

    def __init__(self, name: str, on_complete: Callable[[dict], None], window_s: float = 10.0):
        self.name = name
        self.on_complete = on_complete
        self.window_s = float(window_s)
        self.events: list[Tuple[str, float]] = []  # (label, monotonic_time)
        self.lock = threading.Lock()

    def add_event(self, label: str):
        now = time.monotonic()
        with self.lock:
            self.events.append((label, now))
            cutoff = now - self.window_s
            self.events = [(lab, ts) for (lab, ts) in self.events if ts >= cutoff]
            self._check_patterns(now)

    def _check_patterns(self, now: float):
        labels = [lab for (lab, _) in self.events]
        count_left = labels.count("LEFT_TURN")
        count_right = labels.count("RIGHT_TURN")
        count_back = labels.count("BACK_TO_FRONT")

        # Pattern 1: >= 2 BACK_TO_FRONT  -> TURN_CYCLE by double-return
        if count_back >= 2:
            self.on_complete({
                "helmet": self.name,
                "type": "TURN_CYCLE",
                "subtype": "DOUBLE_RETURN",
                "detail": f"{count_back} BACK_TO_FRONT within {self.window_s:.1f}s",
                "timestamp": round(now, 3)
            })
            self.events.clear()
            return

        # Pattern 2: >= 2 LEFT and >= 2 RIGHT -> TURN_CYCLE by LR mix
        if count_left >= 2 and count_right >= 2:
            self.on_complete({
                "helmet": self.name,
                "type": "TURN_CYCLE",
                "subtype": "LR_MIX",
                "detail": f"{count_left} LEFT_TURN and {count_right} RIGHT_TURN within {self.window_s:.1f}s",
                "timestamp": round(now, 3)
            })
            self.events.clear()
            return


trackers: Dict[str, ActionTracker] = {}


def on_complete_action(action: dict):
    """
    Called when a TURN_CYCLE is detected for a helmet.
    """
    helmet = action["helmet"]
    subtype = action.get("subtype", "")
    detail = action.get("detail", "")
    print("\n=== COMPLETE TURN CYCLE ===")
    print(f"Helmet : {helmet}")
    print(f"Subtype: {subtype}")
    print(f"Detail : {detail}")
    print(f"Time   : {action['timestamp']}")
    print("================================\n")

    if subtype == "DOUBLE_RETURN":
        phrase = f"{helmet} completed turn cycle by double back to front"
    else:
        phrase = f"{helmet} completed turn cycle"

    fusion.maybe_speak(phrase)


In [7]:

# ================================
# helper: last_val from phyphox JSON
# ================================
def _last_val(root: dict, key: str):
    if not isinstance(root, dict):
        return None
    obj = root.get(key)
    if not isinstance(obj, dict):
        return None
    buf = obj.get("buffer")
    if isinstance(buf, list) and buf:
        return buf[-1]
    return None


# ================================
# Unified event handler
# ================================
def handle_event_from_sensor(sensor_name: str, ev: Dict[str, Any]):
    """
    Called whenever a RealTimeHeadTurnDetector instance produces an event.
    ev["label"] ∈ {"LEFT_TURN","RIGHT_TURN","STATIC","BACK_TO_FRONT"}
    """
    print(f"[{sensor_name}] {ev}")
    label = ev["label"]

    # 1) feed ActionTracker
    if sensor_name not in trackers:
        trackers[sensor_name] = ActionTracker(sensor_name, on_complete_action, window_s=10.0)
    trackers[sensor_name].add_event(label)

    # 2) per-event voice feedback
    if label == "LEFT_TURN":
        words = f"{sensor_name} left turn" if SPEAK_SOURCE_NAME else "Left turn"
        fusion.maybe_speak(words)
    elif label == "RIGHT_TURN":
        words = f"{sensor_name} right turn" if SPEAK_SOURCE_NAME else "Right turn"
        fusion.maybe_speak(words)
    elif label == "BACK_TO_FRONT":
        words = f"{sensor_name} facing forward" if SPEAK_SOURCE_NAME else "Facing forward"
        fusion.maybe_speak(words)
    # STATIC usually skipped


In [8]:

# ================================
# Per-sensor streaming loop
# ================================
def run_sensor_loop(name: str, url: str, period_s: float, log_csv: Optional[str] = None):
    detector = RealTimeHeadTurnDetector(on_event=lambda ev: handle_event_from_sensor(name, ev))

    session = requests.Session()
    session.headers.update({"Accept": "application/json"})

    csv_writer = None
    fobj = None
    if log_csv:
        fobj = open(log_csv, "w", newline="")
        csv_writer = csv.writer(fobj)
        csv_writer.writerow(["host_iso_ms", "host_unix_ms", "gyr_time_s", "gyrX", "gyrY", "gyrZ"])

    t0_sensor = None
    t0_host = None

    print(f"[{name}] polling {url} @ {int(1/period_s)} Hz (Ctrl+C to stop)")
    try:
        while True:
            loop_start = time.perf_counter()
            gx = gy = gz = gt = None

            try:
                r = session.get(url, timeout=0.5)
                r.raise_for_status()
                data = r.json()
                root = data.get("buffer", data)
                gx = _last_val(root, "gyrX")
                gy = _last_val(root, "gyrY")
                gz = _last_val(root, "gyrZ")
                gt = _last_val(root, "gyr_time")
            except Exception as e:
                # network hiccup; keep cadence
                print(f"[{name}] HTTP error: {e}")

            # Host timestamps
            now_ns = time.time_ns()
            unix_ms = now_ns // 1_000_000
            iso_ms = dt.datetime.utcfromtimestamp(unix_ms / 1000).isoformat(timespec="milliseconds")

            # logging
            if csv_writer is not None:
                csv_writer.writerow([
                    iso_ms,
                    unix_ms,
                    "" if gt is None else f"{float(gt):.6f}",
                    "" if gx is None else f"{float(gx):.6f}",
                    "" if gy is None else f"{float(gy):.6f}",
                    "" if gz is None else f"{float(gz):.6f}",
                ])
                fobj.flush()

            # feed detector using gyrY (rad/s)
            if gy is not None:
                if gt is not None:
                    if t0_sensor is None:
                        t0_sensor = float(gt)
                    ts = float(gt) - t0_sensor
                else:
                    if t0_host is None:
                        t0_host = time.monotonic()
                    ts = time.monotonic() - t0_host

                try:
                    detector.update(ts, float(gy))
                except Exception as e:
                    print(f"[{name}] detector.update error: {e}")

            # maintain target period
            elapsed = time.perf_counter() - loop_start
            time.sleep(max(0.0, period_s - elapsed))

    except KeyboardInterrupt:
        print(f"\n[{name}] stopped by user.")
    finally:
        detector.flush()
        if fobj is not None:
            fobj.close()


In [9]:

# ================================
# Main
# ================================
if __name__ == "__main__":
    threads = []
    for s in SENSORS:
        th = threading.Thread(
            target=run_sensor_loop,
            args=(s["name"], s["url"], PERIOD_S, s.get("log_csv")),
            daemon=True
        )
        th.start()
        threads.append(th)

    try:
        while any(t.is_alive() for t in threads):
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("\n[Main] stopping…")

[One] polling http://192.168.23.4:8080/get?gyrX&gyr_time&gyrY&gyrZ @ 50 Hz (Ctrl+C to stop)
[Two] polling http://192.168.23.136:8080/get?gyrX&gyr_time&gyrY&gyrZ @ 50 Hz (Ctrl+C to stop)
[Two] {'start_s': 2.963, 'end_s': 6.116, 'duration_s': 3.153, 'label': 'RIGHT_TURN', 'net_angle_deg': -73.03}
[Two] {'start_s': 6.356, 'end_s': 7.337, 'duration_s': 0.981, 'label': 'LEFT_TURN', 'net_angle_deg': 53.47}
[Two] {'start_s': 9.329, 'end_s': 10.24, 'duration_s': 0.911, 'label': 'LEFT_TURN', 'net_angle_deg': 59.51}
[Two] {'start_s': 11.101, 'end_s': 12.372, 'duration_s': 1.271, 'label': 'RIGHT_TURN', 'net_angle_deg': -71.09}

=== COMPLETE TURN CYCLE ===
Helmet : Two
Subtype: LR_MIX
Detail : 2 LEFT_TURN and 2 RIGHT_TURN within 10.0s
Time   : 718366.078

[Two] {'start_s': 16.36, 'end_s': 17.727, 'duration_s': 1.367, 'label': 'RIGHT_TURN', 'net_angle_deg': -41.37}
[Two] {'start_s': 19.298, 'end_s': 20.8, 'duration_s': 1.501, 'label': 'LEFT_TURN', 'net_angle_deg': 48.14}
[Two] {'start_s': 21.65, 'e

[One] {'start_s': 178.13, 'end_s': 178.706, 'duration_s': 0.576, 'label': 'BACK_TO_FRONT', 'net_angle_deg': 6.8}
[Two] {'start_s': 203.001, 'end_s': 204.293, 'duration_s': 1.291, 'label': 'RIGHT_TURN', 'net_angle_deg': -71.82}
[Two] {'start_s': 214.432, 'end_s': 215.253, 'duration_s': 0.821, 'label': 'LEFT_TURN', 'net_angle_deg': 71.15}
[Two] {'start_s': 215.503, 'end_s': 218.496, 'duration_s': 2.993, 'label': 'RIGHT_TURN', 'net_angle_deg': -85.15}
[Two] {'start_s': 220.027, 'end_s': 220.998, 'duration_s': 0.971, 'label': 'LEFT_TURN', 'net_angle_deg': 70.0}
[Two] {'start_s': 221.849, 'end_s': 223.03, 'duration_s': 1.181, 'label': 'RIGHT_TURN', 'net_angle_deg': -75.21}

=== COMPLETE TURN CYCLE ===
Helmet : Two
Subtype: LR_MIX
Detail : 2 LEFT_TURN and 2 RIGHT_TURN within 10.0s
Time   : 718576.734

[Two] {'start_s': 228.856, 'end_s': 229.847, 'duration_s': 0.991, 'label': 'RIGHT_TURN', 'net_angle_deg': -65.16}
[Two] {'start_s': 231.248, 'end_s': 232.239, 'duration_s': 0.991, 'label': 'LEF

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BB7340>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BB7910>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BB7B80>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BB78E0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529848B0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529842B0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529F63D0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852984520>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BCAA90>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

...

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852984580>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529846D0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BB7730>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BCA940>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BCA910>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002085273C160>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852BCA490>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852992460>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529845E0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529925E0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529846A0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529844F0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy

[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208526F7D00>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000020852947AF0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gyrZ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000208529924F0>, 'Connection to 192.168.23.4 timed out. (connect timeout=0.5)'))
[One] HTTP error: HTTPConnectionPool(host='192.168.23.4', port=8080): Max retries exceeded with url: /get?gyrX&gyr_time&gyrY&gy